Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=github%2Froll-points-posita-dyce-11%2Froll_points.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/github/roll-points-posita-dyce-11)]*

## [``dyce``](https://posita.github.io/dyce/) and [Icepool](https://github.com/HighDiceRoller/icepool) solution to [“Count Dice equal to the dice target, and increase dice up to the dice target with total of Roll points mechanic help.”](https://github.com/posita/dyce/discussions/11#discussioncomment-5493832)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce, icepool
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0", "icepool~=0.25.3"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce


try:
    import showit
except ImportError:
    # Work-around for JupyterLite in non-Chromium browsers
    import js
    import os
    from urllib.parse import urljoin, urlparse, urlunparse
    loc_url = urlparse(js.location.toString())
    ext_root = loc_url.path.find("/extensions/@jupyterlite/")
    if ext_root < 0:
        base_url = urljoin(js.location.toString(), "../files/")
    else:
        loc_url = loc_url._replace(path=loc_url.path[:ext_root])
        base_url = urljoin(urlunparse(loc_url), "files/")
    for path in (
                "github/roll-points-posita-dyce-11/showit.py",
            ):
        url = urljoin(base_url, path)
        res = await js.fetch(url)
        assert 200 <= res.status < 300
        text = await res.text()
        with open(os.path.basename(path), "w") as f:
            f.write(text)
    import showit

## First Attempt: “Optimized” Exploded Die

Truly exploding dice can theoretically explode forever.
In this approach, we craft an exploded die just big enough to accommodate the target number and collapse outcomes that have no hope of ever acheiving the target, even with available roll points. Then we subject that die to the core mechanic.

In [2]:
from dyce import H
from dyce.evaluation import explode
d20 = H(20)
DEFAULT_DIE = d20

In [3]:
from dyce import P
from dyce.evaluation import PResult, foreach

def mechanic_w_optimized_die(pool_size: int, roll_points_available: int, target: int, die: H = DEFAULT_DIE) -> H:
    # Sanity checks
    assert target >= 2
    max_die_val = max(die)
    assert max_die_val >= 2
    assert min(die) == 1
    explosions_needed_to_hit_target = target // max_die_val + int(target % max_die_val != 0) - 1
    # Make an "exploded" die just big enough to accommodate the target
    # (exploding dice can be expanded theoretically infinitely)
    exploded_die = explode(die, limit=explosions_needed_to_hit_target)
    assert max(exploded_die) - max_die_val < target <= max(exploded_die)
    # Collapse all outcomes from our exploded die that have no hope of
    # being elevated to the target, even with roll points
    optimized_die = H(((outcome if outcome == 1 or outcome + roll_points_available >= target else 0, count) for outcome, count in exploded_die.items()))
    def _eval(result: PResult):
        roll_points_left = roll_points_available
        result_points = 0
        nat_ones = 0
        # Rolls' outcomes are sorted lowest to highest, but we want to
        # examine them in reverse
        for outcome in result.roll[::-1]:
            if outcome >= target:
                result_points += 1
            elif outcome == 1:
                nat_ones += 1
            elif roll_points_left > 0:
                # We have roll points left, so if we can cover the
                # delta between the current outcome and the target
                # without going negative, we can count this outcome as
                # meeting the target
                roll_points_left -= target - outcome
                if roll_points_left >= 0:
                    result_points += 1
        return result_points if result_points else -nat_ones

    return foreach(_eval, pool_size @ P(optimized_die))

mechanic_w_optimized_die(pool_size=3, roll_points_available=6, target=22)

H({-3: 20, -2: 840, -1: 11760, 0: 54880, 1: 82173, 2: 10016, 3: 311})

## Second Attempt: Explode Inline and Recurse

Another idea is to only explode a die when necessary as part of the core mechanic itself.
This is doable, but complicates the implementation substantially.
Perhaps unsurprisingly, this does not provide a performance improvement over the “optimized” exploded die approach.

In [4]:
def mechanic_w_recursive_inline_explosion(pool_size: int, roll_points_available: int, target: int, die: H = DEFAULT_DIE) -> H:
    assert target >= 2
    assert min(die) == 1
    max_die_val = max(die)
    assert max_die_val >= 2
    explosions_needed_to_hit_target = target // max_die_val + int(target % max_die_val != 0) - 1
    explosion_val_limit = (explosions_needed_to_hit_target + 1) * max_die_val
    assert (explosion_val_limit - max_die_val) < target <= explosion_val_limit
    def _explode_roll(done_part: tuple[int], exploding_part: tuple[int]) -> H:
        def _eval(result: PResult):
            # Sanity checks
            assert len(result.roll) == len(exploding_part)
            # Add the newly-exploded outcomes to the prior values
            exploded_roll = tuple(a + b for a, b in zip(exploding_part, result.roll))
            exploded_done = tuple(outcome for outcome in exploded_roll if outcome >= target or outcome % max_die_val != 0)
            still_exploding_part = tuple(outcome for outcome in exploded_roll if outcome < target and outcome % max_die_val == 0)
            now_done_part = done_part + exploded_done
            if still_exploding_part:
                # Recurse if we still have unexploded dice that can
                # still reach the target
                return _explode_roll(now_done_part, still_exploding_part)
            elif len(now_done_part) == pool_size:
                # We're done exploding, so compute and return the
                # results (similar to mechanic_w_optimized_die
                # above)
                assert len(now_done_part) == pool_size, f"{now_done_part}"
                assert all(outcome <= explosion_val_limit for outcome in done_part)
                roll_points_left = roll_points_available
                result_points = 0
                nat_ones = 0
                for outcome in now_done_part[::-1]:
                    if outcome >= target:
                        result_points += 1
                    elif outcome == 1:
                        nat_ones += 1
                    elif roll_points_left > 0:
                        roll_points_left -= target - outcome
                        if roll_points_left >= 0:
                            result_points += 1
                return result_points if result_points else -nat_ones
            else:
                # We should never be here
                assert False, f"{now_done_part}, {still_exploding_part}"
        
        return foreach(_eval, len(exploding_part) @ P(die), limit=-1)
    
    return _explode_roll((), (0,) * pool_size)

mechanic_w_recursive_inline_explosion(pool_size=3, roll_points_available=6, target=22)

H({-3: 20, -2: 840, -1: 11760, 0: 54880, 1: 82173, 2: 10016, 3: 311})

## Third Attempt: Introducing [Icepool](https://github.com/HighDiceRoller/icepool)

Both of the above approaches are not useful in practice because neither are performant beyond trivially small inputs.
Albert Julius Liu’s Icepool has a learning curve, but where one can wrap one’s head around the computation model, it is great at dealing with lots of dice or dice with lots of sides.
This is particularly helpful with mechanics that involve lots of d20s, like this one.
We use a similar technique as above to create a pre-exploded die that is “optimized” to the target number.

In [5]:
import icepool
from collections import Counter
from dataclasses import dataclass

@dataclass(frozen=True)
class State:
    roll_points_left: int
    nat_ones: int = 0
    result_points: int = 0

class Mechanic(icepool.MultisetEvaluator):
    def __init__(self, roll_points_available: int, target: int):
        self._roll_points_available = roll_points_available
        self._target = target

    def final_outcome(self, final_state) -> int:
        return final_state.result_points if final_state.result_points > 0 else -final_state.nat_ones

    def next_state(self, state, outcome, count):
        if state is None:
            state = State(roll_points_left=self._roll_points_available)
        if outcome >= self._target:
            state = State(
                roll_points_left=state.roll_points_left,
                nat_ones=state.nat_ones,
                result_points=state.result_points + count,
            )
        elif outcome == 1:
            state = State(
                roll_points_left=state.roll_points_left,
                nat_ones=state.nat_ones + count,
                result_points=state.result_points,
            )
        else:
            diff_to_target = self._target - outcome
            new_result_points = min(state.roll_points_left // diff_to_target, count)
            state = State(
                roll_points_left=state.roll_points_left - new_result_points * diff_to_target,
                nat_ones=state.nat_ones,
                result_points=state.result_points + new_result_points,
            )
        return state

    def order(self, *_) -> icepool.Order:
        return icepool.Order.Descending

def _mechanic_icepool(pool_size: int, roll_points_available: int, target: int, die: icepool.Die) -> icepool.Die:
    assert target >= 2
    assert min(die) == 1
    max_die_val = max(die)
    assert max_die_val >= 2
    explosions_needed_to_hit_target = target // max_die_val + int(target % max_die_val != 0) - 1
    exploded_die = die.explode(depth=explosions_needed_to_hit_target)
    assert max(exploded_die) - max_die_val < target <= max(exploded_die)
    optimized_outcome_counts = Counter()
    for outcome, count in exploded_die.items():
        optimized_outcome_counts[outcome if outcome == 1 or outcome + roll_points_available >= target else 0] += count
    optimized_die = icepool.Die(optimized_outcome_counts)
    mechanic = Mechanic(roll_points_available, target)
    return mechanic(optimized_die.pool(pool_size))

def mechanic_icepool_dyce_wrapper(pool_size: int, roll_points_available: int, target: int, die: H = DEFAULT_DIE) -> H:
    return H(_mechanic_icepool(pool_size, roll_points_available, target, die=icepool.Die(die))).lowest_terms()

mechanic_icepool_dyce_wrapper(pool_size=3, roll_points_available=6, target=22)

H({-3: 20, -2: 840, -1: 11760, 0: 54880, 1: 82173, 2: 10016, 3: 311})

## Visualization

We also easily translate from Icepool to `dyce` primitives for use with [`anydyce`](https://github.com/posita/anydyce)’s visualization tools.

In [6]:
from showit import showit
showit(mechanic_icepool_dyce_wrapper)

Output()

In [7]:
# Time comparisons for various approaches
for pool_size, roll_points_available, target in (
    (1, 4, 10),
    (2, 6, 12),
    (3, 8, 14),
    (4, 10, 16),
    (3, 6, 22),
):
    print()
    res_w_optimized_die = mechanic_w_optimized_die(pool_size, roll_points_available, target)
    res_w_recursive_inline_explosion = mechanic_w_recursive_inline_explosion(pool_size, roll_points_available, target)
    res_icepool = mechanic_icepool_dyce_wrapper(pool_size, roll_points_available, target)
    assert res_w_recursive_inline_explosion == res_w_optimized_die
    assert res_icepool == res_w_recursive_inline_explosion
    print(f"mechanic_...({pool_size}, {roll_points_available}, {target}):")
    print(res_icepool.format())
    %timeit mechanic_w_optimized_die(pool_size, roll_points_available, target)
    %timeit mechanic_w_recursive_inline_explosion(pool_size, roll_points_available, target)
    %timeit mechanic_icepool_dyce_wrapper(pool_size, roll_points_available, target)


mechanic_...(1, 4, 10):
avg |    0.70
std |    0.56
var |    0.31
 -1 |   5.00% |##
  0 |  20.00% |##########
  1 |  75.00% |#####################################
339 µs ± 4.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
423 µs ± 5.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
451 µs ± 7.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

mechanic_...(2, 6, 12):
avg |    1.42
std |    0.69
var |    0.47
 -2 |   0.25% |
 -1 |   2.00% |#
  0 |   4.00% |##
  1 |  42.75% |#####################
  2 |  51.00% |#########################
1.23 ms ± 9.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.92 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.06 ms ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

mechanic_...(3, 8, 14):
avg |    2.00
std |    0.78
var |    0.61
 -3 |   0.01% |
 -2 |   0.15% |
 -1 |   0.60% |
  0 |   0.80% |
  1 |  22.74% |###########
  2 |  48.66% |########################
 